<a href="https://colab.research.google.com/github/toshimitsu10432/tut_B3_data_informatics/blob/main/baseline2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!kill -9 $(lsof -t)

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import AlexNet, alexnet, resnet18, resnet50, efficientnet_b3, efficientnet_b7
import torch.nn.functional as nF
import torchvision.transforms.functional as tF
from torch.optim.optimizer import Optimizer
import torchvision
import albumentations as A
import cv2
import time

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
%cd ./drive/MyDrive/B3_emotion/

[Errno 2] No such file or directory: './drive/MyDrive/B3_emotion/'
/content/drive/MyDrive/B3_emotion


In [35]:
!ls

Alexnet_weight.pth  Emotion6		 sample_submission.csv
baseline.ipynb	    emotion.ipynb	 submit2.csv
csv_emotion6	    resnet18_weight.pth  submit.csv


In [36]:
train = pd.read_csv('./csv_emotion6/ground_truth_train.csv')
test = pd.read_csv('./csv_emotion6/ground_truth_test.csv')

In [37]:
train.head()

,image,label
0,disgust/1.jpg,2
1,surprise/1.jpg,4
2,fear/1.jpg,3
3,joy/1.jpg,4
4,sadness/1.jpg,5


In [38]:
test.head()

,image
0,anger/230.jpg
1,disgust/245.jpg
2,disgust/249.jpg
3,joy/253.jpg
4,disgust/255.jpg


In [39]:
train["label"] = train["label"]-1

In [82]:
class Dataset(Dataset):
    def __init__(self, df, phase, transform=None, albu_transform=None):
        self.df = df
        self.phase = phase
        self.transform = transform
        self.albu_transform = albu_transform
        self.emotion_path = "./Emotion6"
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img = self.pull_item(index)
        if self.phase=="train":
          tmp = self.albu_transform(image=img)
          img = tmp['image']
          img = self.transform(img)
          return img, self.df["label"].iloc[index]
        elif self.phase=="test":
          img = self.transform(img)
          return img
        
    def pull_item(self, index):
        img_path = os.path.join(self.emotion_path, self.df["image"].iloc[index])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img

In [83]:
train["image"][0]

'disgust/1.jpg'

In [84]:
print(len(train), len(test))

1587 393


In [85]:
print(len(train[:1200]), len(train[1200:]))

1200 387


In [86]:
print(torch.cuda.is_available())

True


In [87]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [88]:
torch.cuda.memory_reserved(device=device)
torch.cuda.empty_cache()
!nvidia-smi

Wed Jan 18 14:36:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    33W /  70W |   1880MiB / 15109MiB |     80%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [89]:
batch_size=20
# データ拡張
train_transformer = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.RandomResizedCrop(300),
                            transforms.RandomHorizontalFlip(),
                            transforms.RandomVerticalFlip(),
                            #transforms.RandomGrayscale(),
                            transforms.RandomRotation(10),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                            ])
albu_transformer = A.Compose([
                            A.GlassBlur(sigma=2, p=0.5),
                            A.ChannelShuffle(p=0.5),
                            A.RGBShift(p=0.5),
                            A.Posterize(p=0.5),
                            A.ToGray(p=0.5),
                            A.RandomBrightnessContrast(p=0.5),
                            A.CLAHE(p=0.5),
                            A.CoarseDropout(p=0.5)
])
test_transformer = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Resize((300, 300)),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                            ])
train_dataset = Dataset(train[:1200], "train", transform=train_transformer, albu_transform=albu_transformer)
val_dataset = Dataset(train[1200:], "train", transform=test_transformer, albu_transform=None)
test_dataset = Dataset(test, "test", transform=test_transformer, albu_transform=None)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader =DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [90]:
"""
for imgs, labels in train_dataloader:
    for img in imgs:
        print(img.shape)
"""

'\nfor imgs, labels in train_dataloader:\n    for img in imgs:\n        print(img.shape)\n'

In [91]:
# モデル定義
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
#model = AlexNet()

In [92]:
#model = alexnet(pretrained=False, num_classes=7)
#model = resnet50(pretrained=False, num_classes=7).to(device)
model = efficientnet_b3(pretrained=True, num_classes=7).to(device)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [93]:
!nvidia-smi

Wed Jan 18 14:36:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    32W /  70W |   1894MiB / 15109MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [94]:
class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=2,reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, input, target):

        ce_loss = nF.cross_entropy(input, target,reduction=self.reduction,weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

In [95]:
tmp = train['label'].value_counts().max()/train['label'].value_counts()
tmp = tmp.astype('float')
tmp = [20.72, 2.629442, 1.962121, 1, 2.149378, 6.317073, 1.992308]

In [96]:
# 損失関数
weights = torch.tensor(tmp).cuda()
criterion = nn.CrossEntropyLoss(weight=weights)
#criterion = FocalLoss(weight=weights)

# 最適化手法
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)

In [97]:
train_acc_list, val_loss_list, val_acc_list = [], [], []
num_epoch=200
start_time = time.time()
for epoch in range(num_epoch):
    train_loss, train_acc, val_loss, val_acc = 0, 0, 0, 0
    model.train()
    for imgs, labels in train_dataloader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        train_acc += (outputs.max(1)[1] == labels).sum().item()
        loss.backward()
        optimizer.step()
    avg_train_loss = train_loss / len(train_dataloader.dataset)
    avg_train_acc = train_acc / len(train_dataloader.dataset)
    
    # ====== val_mode ======
    model.eval()
    with torch.no_grad():
        for imgs, labels in val_dataloader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_acc += (outputs.max(1)[1] == labels).sum().item()
    avg_val_loss = val_loss / len(val_dataloader.dataset)
    avg_val_acc = val_acc / len(val_dataloader.dataset)
    
    print("epoch:{}".format(epoch))
    print("avg_train_loss:{}　avg_train_acc:{}".format(avg_train_loss, avg_train_acc)) #0.355 0.3091
    print("avg_val_loss:{} avg_val_acc:{}".format(avg_val_loss, avg_val_acc))
end_time = time.time()
print(end_time-start_time)
torch.save(model.state_dict(), 'resnet18_weight.pth')

TypeError: ignored

In [ ]:
outputs[0]

In [ ]:
torch.save(model.state_dict(), 'resnet18_weight.pth')

In [ ]:
model = AlexNet()
model.load_state_dict(torch.load('Alexnet_weight.pth'))

In [ ]:
model.eval()
predicts = []
with torch.no_grad():
    for img in test_dataloader:
        img = img.to(device)
        predict = model(img)
        predicts.append(predict.argmax().detach().cpu().numpy())

In [ ]:
sample_submission = pd.read_csv('./sample_submission.csv')
#sample_submission["label"] = predicts
sample_submission['label'] = pd.Series(predicts, dtype='int')

In [ ]:
sample_submission = sample_submission[['image', 'label']]

In [ ]:
sample_submission.to_csv('./submit.csv', index=False)